In [ ]:
# Funkcja odpalana jako pandas_udf




In [ ]:
from abc import ABC, abstractmethod
from typing import List



class RetrainingStrategy(ABC):
    
    @abstractmethod
    def get_retraining_data(x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        pass

In [ ]:
from abc import ABC, abstractmethod
from typing import List


class EvaluationStrategyManager(ABC):
    
    @abstractmethod
    def get_curr_ref_data(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        pass

In [ ]:
from abc import ABC, abstractmethod
from typing import List


class ModelEstimatorPipeline(ABC):
    
    @abstractmethod
    def handle(self, x, y):
        pass

    @abstractmethod
    def adjust_model(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        pass

    @abstractmethod
    def get_name(self):
        pass

In [ ]:
from sklearn.pipeline import Pipeline
from typing import Dict, List



class ModelSklearnPipeline(ModelEstimatorPipeline):

    def __init__(self, sklearn_pipeline: Pipeline, hyperparameter_space: Dict, retraining_strategy: TrainingStrategyManager):
        self.estimator = sklearn_pipeline
        self.hyperparameter_space = hyperparameter_space
        self.retraining_strategy = retraining_strategy

    def handle(self, x, y):
        return self.estimator.predict(x)
    
    def adjust_model(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        x_train, y_train = self.retraining_strategy.get_retraining_data(x_history, y_history, prediction_history, drift_history)
        self.estimator.fit(x_train, y_train) 

    def get_name(self):
        return super().get_name() # TO DO

In [ ]:
from river import compose
from typing import List


class ModelRiverPipeline(ModelEstimatorPipeline):
    
    def __init__(self, river_pipeline: compose.Pipeline):
        self.estimator = river_pipeline

    def handle(self, x, y):
        prediction = self.estimator.predict_one(x)
        self.estimator.learn_one(x, y)
        return prediction

    def adjust_model(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]):
        return
    
    def get_name(self):
        return super().get_name() # TO DO

In [ ]:
class ModelEvaluationPipeline:
    
    def __init__(self, metric_steps):
        self.metric_steps = metric_steps

    def handle(self, y_true, y_predict):
        results = {}
        for metric_name, metric in self.steps:
            metric_value = metric.update(y_true, y_predict)
            results.update({metric_name: metric_value})
        return results

In [ ]:
from abc import ABC, abstractmethod
from typing import List



class MonitoringStep(ABC):

    @abstractmethod
    def monitor(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]) -> bool:
        pass

In [ ]:
from evidently.test_suite import TestSuite



class EvidentlyMonitoringStep(MonitoringStep):

    def __init__(self, evidently_test_suite: TestSuite, evaluation_strategy: EvaluationStrategyManager):
        self.detector = evidently_test_suite
        self.eval_strategy = evaluation_strategy

    def monitor(self, x_history: List, y_history: List[int], prediction_history: List[int], drift_history: List[int]) -> bool:
        curr, ref = self.eval_strategy.get_curr_ref_data(x_history, y_history, prediction_history, drift_history)
        self.detector.run(reference_data=ref,current_data=curr)
        report = self.detector.as_dict()
        return True # to do based on report
        

In [ ]:

# Experiment -> uruchomiony na danym partition: definiuje experiment pipeline



datastream_name = datastream['name'][0]
datastream = datastream.drop(col=['name'])
logger = Logger(dataset_name=name)

logger.start()
for x, y in datastream:

   logger.iter()
   pipe = StreamClassificationPipeline()

   pipe.handle(x, y)
   logger.iter_end()

logger.end()

        


In [ ]:
list([{'a': 4}][0].values())[0]

In [ ]:
from river.drift import ADWIN

a = ADWIN()

In [ ]:
type(a)

In [ ]:
a.clock

In [ ]:
'_helper'.startswith('_')

In [ ]:
vars(a).items()

In [ ]:
{item for item in vars(a).items() if not item[0].startswith('_')}

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets, ensemble, model_selection

from evidently import ColumnMapping
from evidently.test_suite import TestSuite

from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_preset import DataQualityTestPreset
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import DataDriftTestPreset
from evidently.test_preset import RegressionTestPreset
from evidently.test_preset import MulticlassClassificationTestPreset
from evidently.test_preset import BinaryClassificationTopKTestPreset
from evidently.test_preset import BinaryClassificationTestPreset

from evidently.tests import TestNumberOfEmptyRows, TestNumberOfEmptyColumns, TestNumberOfDuplicatedRows, TestNumberOfDuplicatedColumns, TestNumberOfDriftedColumns, TestShareOfDriftedColumns

In [ ]:
#Dataset for Data Quality and Integrity
adult_data = datasets.fetch_openml(name='adult', version=2, as_frame='auto')
adult = adult_data.frame

adult_ref = adult[~adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]
adult_cur = adult[adult.education.isin(['Some-college', 'HS-grad', 'Bachelors'])]

adult_cur.iloc[:2000, 3:5] = np.nan

In [ ]:
#Dataset for Regression
housing_data = datasets.fetch_california_housing(as_frame='auto')
housing = housing_data.frame

housing.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing['prediction'] = housing_data['target'].values + np.random.normal(0, 3, housing.shape[0])

housing_ref = housing.sample(n=5000, replace=False)
housing_cur = housing.sample(n=5000, replace=False)

In [ ]:
#Dataset for Binary Probabilistic Classifcation
bcancer_data = datasets.load_breast_cancer(as_frame='auto')
bcancer = bcancer_data.frame

bcancer_ref = bcancer.sample(n=300, replace=False)
bcancer_cur = bcancer.sample(n=200, replace=False)

bcancer_label_ref = bcancer_ref.copy(deep=True)
bcancer_label_cur = bcancer_cur.copy(deep=True)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=10)
model.fit(bcancer_ref[bcancer_data.feature_names.tolist()], bcancer_ref.target)

bcancer_ref['prediction'] = model.predict_proba(bcancer_ref[bcancer_data.feature_names.tolist()])[:, 1]
bcancer_cur['prediction'] = model.predict_proba(bcancer_cur[bcancer_data.feature_names.tolist()])[:, 1]

bcancer_label_ref['prediction'] = model.predict(bcancer_label_ref[bcancer_data.feature_names.tolist()])
bcancer_label_cur['prediction'] = model.predict(bcancer_label_cur[bcancer_data.feature_names.tolist()])

In [ ]:
#Dataset for Multiclass Classifcation
iris_data = datasets.load_iris(as_frame='auto')
iris = iris_data.frame

iris_ref = iris.sample(n=75, replace=False)
iris_cur = iris.sample(n=75, replace=False)

model = ensemble.RandomForestClassifier(random_state=1, n_estimators=3)
model.fit(iris_ref[iris_data.feature_names], iris_ref.target)

iris_ref['prediction'] = model.predict(iris_ref[iris_data.feature_names])
iris_cur['prediction'] = model.predict(iris_cur[iris_data.feature_names])

In [ ]:
data_drift_dataset_tests = TestSuite(tests=[
    TestNumberOfEmptyRows(),
    TestNumberOfEmptyColumns(),
    TestNumberOfDuplicatedRows(),
    TestNumberOfDuplicatedColumns(),
    TestNumberOfDriftedColumns(),
    TestShareOfDriftedColumns(),
])

data_drift_dataset_tests.run(reference_data=adult_ref, current_data=adult_cur)

In [ ]:
#test preset as a python object
res = data_drift_dataset_tests.as_dict()

In [ ]:
res['tests']

In [ ]:
is_drift_detected = False
detection_idx = 3



In [ ]:
desc

In [ ]:
from collections import Counter


x = Counter({'SUCCESS': 3, 'FAIL': 3})

In [ ]:
x.elements()

In [ ]:
def foo():
    return 1, 2, {}

In [ ]:
a, b, kwargs = foo()

In [ ]:
class Foo:

    def __init__(self, a: int, b: int, weight: int=4, height: int=3):
        self.a = a
        self.b = b
        self.weight = weight
        self.height = height


In [ ]:
f= Foo(a, b, **kwargs)

In [ ]:
f.a

In [ ]:
f.weight

In [ ]:
f.c

In [ ]:
from collections import Counter

In [ ]:
d = {
            'tests': [{
                'name': 'self.step_name',
                'description': f'Drift detected at idx.: {3}',
                'status': 'FAIL',
                'group': 'river-detector',
                'parameters': {
                    'detected_at_idx': 3,
                    'detector_type': 'type(self.detector)'
                }.update({'item for item in vars(self.detector).items()' : "if not item[0].startswith('_')"})
            }],
            'summary': {
                'all_passed': 0,
                'total_tests': 1,
                'success_tests': 0,
                'failed_tests': 1,
                'by_status': Counter({'SUCCESS': 0, 'FAIL': 1})}
        }

In [ ]:
any_failed = d['summary']['failed_tests'] > 0
all_failed = d['summary']['failed_tests'] == d['summary']['total_tests']

In [ ]:
d

In [ ]:
any_failed

In [ ]:
all_failed

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.random.random(size=(10, 30)))

In [ ]:
df.head()

In [ ]:
n_curr = 5
n_ref = 3

In [ ]:
df.iloc[-n_curr:,:]

In [ ]:
df.iloc[-(n_curr + n_ref):-n_curr,:]

In [ ]:
s = [1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
s[3:]

In [ ]:
data_path = './data/ADS/internet_ads.arff.zip'

In [ ]:
from utils.read_data.internet_ads import get_internet_ads_df

In [ ]:
df = get_internet_ads_df()

In [ ]:
from batchstream.pipelines.stream.model_river_pipeline import ModelRiverPipeline
from batchstream.evaluation.model_evaluation_pipeline import ModelEvaluationPipeline
from river.metrics import Accuracy, ROCAUC

In [ ]:
from river import linear_model
from river import preprocessing
from batchstream.utils.logging.performance_logger import PerformanceEvalLogger

scaler = preprocessing.StandardScaler()
log_reg = linear_model.LogisticRegression()
river_model = scaler | log_reg
perf_logger = PerformanceEvalLogger('test-xgrhtueifj')

In [ ]:
stream_pipeline = ModelRiverPipeline(river_model)

acc = Accuracy()
roc_auc = ROCAUC()
pipeline_evaluation = ModelEvaluationPipeline(metric_steps=[('roc_auc', roc_auc)])



In [ ]:
from batchstream.experiment.experiment import StreamExperiment


In [ ]:
exp1 = StreamExperiment(stream_pipeline, pipeline_evaluation, perf_logger)

In [ ]:
exp1.run(df)

In [ ]:
report_batch = [
        {'accuracy': 0.0, 'roc_auc': -0.0},
        {'accuracy': 0.5, 'roc_auc': 0.5},
        {'accuracy': 0.33, 'roc_auc': 0.5},
        {'accuracy': 0.5, 'roc_auc': 0.5},
        {'accuracy': 0.6, 'roc_auc': 0.67},
        {'accuracy': 0.67, 'roc_auc': 0.67},
        {'accuracy': 0.71, 'roc_auc': 0.75},
        {'accuracy': 0.75, 'roc_auc': 0.75}
    ]

In [ ]:
perf_logger.log_eval_report(report_batch)

In [ ]:
report_batch_2 = [
        {'accuracy': 0.75, 'roc_auc': -0.0},
        {'accuracy': 0.71, 'roc_auc': 0.5},
        {'accuracy': 0.67, 'roc_auc': 0.5},
        {'accuracy': 0.6, 'roc_auc': 0.5},
        {'accuracy': 0.5, 'roc_auc': 0.67},
        {'accuracy': 0.4, 'roc_auc': 0.67},
        {'accuracy': 0.33, 'roc_auc': 0.75},
        {'accuracy': 0.2, 'roc_auc': 0.75}
    ]

In [ ]:
perf_logger.log_eval_report(report_batch_2)

In [ ]:
perf_logger.log_info("Start evaluation")

In [ ]:
perf_logger.log_info("First batch logged.")

In [ ]:
perf_logger.log_info("Second batch logged.")

In [ ]:
perf_logger.log_info("End evaluation")

In [ ]:
import pandas as pd


In [ ]:
report_artifact = pd.read_csv('./out/1111/1111_performance_eval_report.csv')

In [ ]:
with open("./log/1111/1111_performance_eval.log", 'r') as fp:
    for count, line in enumerate(fp):
        pass
return count + 1

In [ ]:
import shutil

shutil.rmtree(f'./log/test_experiment')
shutil.rmtree(f'./out/test_experiment')

In [ ]:
import logging

In [ ]:
logging.shutdown()

In [1]:

from batchstream.utils.logging.base.logger_factory import LoggerFactory
    

